# Crop Analysis Notebook

Welcome to the Weather Data Visualization Notebook! This notebook is designed to help you fetch and visualize weather data for the past N (N>0) days.

## Setup Instructions

Before you start using this notebook, please follow these steps to set up the Conda environment:

1. **Create and Activate the Conda Environment**:
   
   Ensure you have Conda installed. Open a terminal and navigate to the root directory of the repository. Run the following command to create a new Conda environment based on the `agro_eye_notebooks.yaml` file located in the `conda` folder:

   ```bash
   conda env create -f conda/agro_eye_notebooks.yaml
   ```
### Install env variable

Put the following env variables in your `~/.env`:
- "URL_OPEN-METEO"
- "SERVICE_ACCOUNT"

```bash
SERVICE_ACCOUNT=my-service-account@...gserviceaccount.com
URL_OPEN-METEO=https://open-meteo.com/
```

In [1]:
# Import the necessary libraries
import requests
import json
from dotenv import load_dotenv
import os
import openmeteo_requests
import requests_cache
import numpy as np
from retry_requests import retry
import matplotlib as mpl
from matplotlib import pyplot as plt
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import ee
import geemap
from geopy.geocoders import Nominatim
# allow images to display in the notebook
from IPython.display import Image, display, HTML
import ipywidgets as widgets
import ipyleaflet
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from ipywidgets import VBox, HBox
import time

In [2]:
load_dotenv(dotenv_path=os.path.expanduser("~/.env"))

True

In [3]:
default_env = {
    "URL_OPEN-METEO", 
    "SERVICE_ACCOUNT"
}

In [4]:
for env in default_env:
    if env not in os.environ:
        raise Exception(f"Missing environment variable: {env}. Please add it to your ~/.env file")

### Define the input variables:
- Start date of the analysis
- End date of the analysis
- Address of the analysis
    - which'll be geocoded to get Lat and Long

In [23]:
# Define the start and end date of the analysis
start_date = "2024-06-01"
end_date = "2024-07-25"
current_date = time.strftime("%Y-%m-%d")

# Check if the end data is lower than the start date and currrent date
if end_date < start_date:
    raise Exception("End date must be greater than start date")

if current_date < end_date:
    raise Exception("End date must be lower than the current date")

In [28]:
# Function to geocode an address and get lat and lon
def geocode(address: str, attemps: int, sleep_time: int = 1) -> tuple:

    for attempt in range(attemps):
        # try to geocode the address
        try:
            geolocator = Nominatim(user_agent="agro-eye")
            location = geolocator.geocode(address)

            print(f'Address "{address}" geocoded successfully!')

            # check if we got a valid location
            if location:
                return location.latitude, location.longitude
            else:
                print(
                    f"Attempt {attempt + 1} failed, trying again in {sleep_time} seconds..."
                )
                time.sleep(sleep_time)

        except Exception as e:
            print(
                f"Attempt {attempt + 1} failed, trying again in {sleep_time} seconds..."
            )
            time.sleep(sleep_time)
            location = geocode(address, attemps, sleep_time)
            return location


# Import the satellite data from Google Earth Engine for the defined study area and the
# specified start and end date
lat, lon = geocode("Rome, Italy", attemps=3)

Address "Rome, Italy" geocoded successfully!


Fetch historical Weather Forcast. The information has been fecthed from [here](https://open-meteo.com/en/docs#hourly=&daily=weather_code,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,daylight_duration,uv_index_max,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&start_date=2024-07-01&end_date=2024-07-09&time_mode=time_interval)

In [29]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

#### Query for daily information

In [30]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "daily": [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "rain_sum",
        "snowfall_sum",
        "precipitation_hours",
        "wind_speed_10m_max",
        "shortwave_radiation_sum",
        "et0_fao_evapotranspiration",
    ],
}
# try to get the weather data from the Open-Meteo API
responses = openmeteo.weather_api(os.environ["URL_OPEN-METEO"], params=params)

In [31]:

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 41.8629150390625°N 12.539912223815918°E
Elevation 42.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [32]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(5).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(6).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(7).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(8).ValuesAsNumpy()

In [33]:
# Create a pandas DataFrame with the daily data
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}


daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration


daily_data = pd.DataFrame(data = daily_data)

In [34]:
daily_data.head()

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2024-06-01 00:00:00+00:00,25.088999,14.089000,0.0,0.0,0.0,0.0,18.584511,28.020000,5.488565
1,2024-06-02 00:00:00+00:00,22.088999,14.339000,8.0,8.0,0.0,8.0,8.942214,14.470000,2.665000
2,2024-06-03 00:00:00+00:00,23.989000,14.188999,0.0,0.0,0.0,0.0,14.205182,26.379999,4.742328
3,2024-06-04 00:00:00+00:00,25.789000,13.488999,0.0,0.0,0.0,0.0,16.700275,25.660000,4.932992
4,2024-06-05 00:00:00+00:00,26.639000,14.539000,0.0,0.0,0.0,0.0,15.893093,28.350000,5.400963


In [36]:
fig = go.Figure()

# Create a subplot layout: 4 rows, 1 column, shared x-axis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Add the max temperature line with hover info
fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['temperature_2m_max'].round(2),
                         mode='lines+markers',  # Added markers for better visibility on hover
                         name='Max Temperature',
                         line=dict(color='red', width=3),
                         hoverinfo='x+y',  # Shows both date and temperature on hover
                         hovertemplate='Date: %{x}<br>MaxTemperature: %{y}°C<extra></extra>',  # Custom hover text
                         showlegend=True  # Show the legend
                        ), row=1, col=1)

# Add the min temperature line with hover info
fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['temperature_2m_min'].round(2),
                         mode='lines+markers',  # Added markers for better visibility on hover
                         name='Min Temperature',
                         line=dict(color='blue', width=3),
                         hoverinfo='x+y',  # Shows both date and temperature on hover
                         hovertemplate='Date: %{x}<br>Min Temperature: %{y}°C<extra></extra>',  # Custom hover text
                         showlegend=True  # Show the legend
                        ), row=1, col=1)


fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['precipitation_sum'],
                         mode='lines+markers',
                         name='Precipitation Sum',
                         line=dict(color='brown', width=3),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Precipitation: %{y}mm<extra></extra>'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['wind_speed_10m_max'].round(2),
                         mode='lines+markers',
                         name='Wind Speed Max (10m)',
                         line=dict(color='green', width=3),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Wind Speed: %{y}Km/h<extra></extra>'),
              row=3, col=1)

# Update y-axes titles
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Temperature Min & Max (°C)", title_font=dict(size=18, weight='bold'), row=1, col=1)
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Precipitations Sum (mm)", title_font=dict(size=18, weight='bold'), row=2, col=1)
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Max Wind Speed (km/h)", title_font=dict(size=18, weight='bold'), row=3, col=1)
fig.update_xaxes(tickfont=dict(size=12, weight='bold'), row=3, col=1)

# Update layout (optional adjustments)
fig.update_layout(height=1000, width=1400, title_text="Weather Forecast Daily", title_font=dict(size=30, weight='bold'), title_x=0.5)

# Show the plot
fig.show()

In [ ]:
#fig.write_image("../images/weather_forecast_daily.png")

#### Query for hourly information 

In [44]:
params_hourly = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": [
        "temperature_2m",
        "relative_humidity_2m",
        "precipitation",
        "rain",
        "snowfall",
        "cloud_cover",
        "et0_fao_evapotranspiration",
        "vapour_pressure_deficit",
        "wind_speed_10m",
        "soil_temperature_0_to_7cm",
        "soil_temperature_7_to_28cm",
        "soil_temperature_28_to_100cm",
        "soil_moisture_0_to_7cm",
        "soil_moisture_7_to_28cm",
        "is_day",
    ],
}

In [45]:
responses = openmeteo.weather_api(os.environ["URL_OPEN-METEO"], params=params_hourly)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 41.8629150390625°N 12.539912223815918°E
Elevation 42.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [46]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(6).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(12).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(13).ValuesAsNumpy()
hourly_is_day = hourly.Variables(14).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    )
}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["is_day"] = hourly_is_day

In [47]:
hourly_dataframe = pd.DataFrame(data = hourly_data)

In [48]:
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,cloud_cover,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,is_day
0,2024-06-01 00:00:00+00:00,15.938999,83.734444,0.0,0.0,0.0,27.0,0.012230,0.294825,12.538134,18.039000,20.588999,18.688999,0.216,0.232,0.0
1,2024-06-01 01:00:00+00:00,14.488999,87.218773,0.0,0.0,0.0,1.8,0.002590,0.211097,10.691453,16.989000,20.338999,18.688999,0.214,0.231,0.0
2,2024-06-01 02:00:00+00:00,14.139000,85.479065,0.0,0.0,0.0,0.9,0.003304,0.234475,9.793059,16.588999,20.088999,18.688999,0.214,0.231,0.0
3,2024-06-01 03:00:00+00:00,14.089000,79.992592,0.0,0.0,0.0,3.6,0.008566,0.322039,8.942214,16.188999,19.838999,18.688999,0.214,0.231,0.0
4,2024-06-01 04:00:00+00:00,14.339000,75.874245,0.0,0.0,0.0,13.5,0.038347,0.394666,9.686609,15.938999,19.639000,18.688999,0.214,0.231,1.0


In [49]:
fig = go.Figure()

# Create a subplot layout: 4 rows, 1 column, shared x-axis
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Add the temperature trace to the first subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['temperature_2m'].round(2),
                         mode='lines+markers',
                         name='Temperature',
                         line=dict(color='red', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Temperature: %{y}°C<extra></extra>'),
              row=1, col=1)

# Add the humidity trace to the second subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['relative_humidity_2m'],
                         mode='lines+markers',
                         name='Humidity',
                         line=dict(color='brown', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Humidity: %{y}%<extra></extra>'),
              row=2, col=1)

# Add the wind speed trace to the third subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['wind_speed_10m'].round(2),
                         mode='lines+markers',
                         name='Wind Speed',
                         line=dict(color='green', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Wind Speed:%{y} Km/h <extra></extra>'),
              row=3, col=1)

# Add the precipitation trace to the third subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['precipitation'].round(2),
                         mode='lines+markers',
                         name='Precipitations',
                         line=dict(color='blue', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Precipitation:%{y} mm <extra></extra>'),
              row=4, col=1)

#offset of the figure with respect to the point
fixed_offset = 3

# Iterate through the dataframe and add annotations
for index, row in hourly_dataframe.iterrows():
    is_day = row['is_day']
    # Determine if it's day (6 AM to 6 PM) or night
    if is_day == 1:
        symbol = '☀️'  # Sun symbol for day
    else:
        symbol = '🌑'  # Moon symbol for night
    
    # Add annotation for each point
    fig.add_annotation(x=row['date'], y=round(row['temperature_2m'], 2) + 2,  # Adjust Y to place the symbol above the point
                       text=symbol,
                       showarrow=False,
                       font=dict(size=10),  # Adjust font size as needed
                       row=1, col=1)

# Update y-axes titles
fig.update_yaxes(title_text="Temperature (°C)", title_font=dict(weight='bold'), row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", title_font=dict(weight='bold'), row=2, col=1)
fig.update_yaxes(title_text="Wind Speed (Km/h)", title_font=dict(weight='bold'), row=3, col=1)
fig.update_yaxes(title_text="Precipitations (mm)", title_font=dict(weight='bold'), row=4, col=1)

# Update layout (optional adjustments)
fig.update_layout(height=800, width=1200, title_text="Weather Forecast Hourly", title_font=dict(size=30, weight='bold'), title_x=0.5)

# Show the plot
fig.show()

In [ ]:
#fig.write_html("../images/weather_forecast_hourly.html")

### Compute the crop analysis

Gather satellite images from the `Sentinel-2`, from 2 input dates and a coordinate point, and from those images generate all the crop metrics.

First it is necessary to set up a Cloud Project, see instruction [here](https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup)

Also to smooth the auth for the Google Earth Engine, and to be future proof in case of an app deployment, is it possible to configure a [Service Account](https://developers.google.com/earth-engine/guides/service_account). 

Once the Service Account is created, is possible to create KEY, using a JSON format. The key file is a special file that allows programs to access Google APIs on behalf of your service account. Ensure it is not possible for anyone to gain unauthorized access to it, since they would be able to access Google APIs on your behalf.

In [58]:
# import Google earth engine module
service_account = os.environ["SERVICE_ACCOUNT"]
credentials = ee.ServiceAccountCredentials(
    service_account, "../.agro-eye-daa28181e1a7.json"
)
ee.Initialize(credentials, project="agro-eye")

Create an interactive map

Once the interactive map is created, to proceed with the code, is necessary to define a Polygon geometry, see example below:

![Capture Polygon](../images/Capture_Polygon.PNG)

After you clicked on the *Draw a Polygon* widget, you can define your polygons of interest, which be the base of the analysis.

![Drawn Polygon](../images/Capture_Drawn_Polygon.PNG)

In [86]:
if lat is None or lon is None:
    raise Exception("Invalid coordinates! Please check the address and try again.")
else:
    # toolbar_ctrl = False removes the toolbar from the Map
    Map = geemap.Map(center=[lat, lon], zoom=12, toolbar_ctrl=False)
    Map.add_layer_control("topright")
    Map.add_basemap("HYBRID")
Map

Map(center=[41.8933203, 12.4829321], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [87]:
# Define a ee.Geometry object of your polygon
geometries_list = []

if len(Map.draw_features) == 0:
    print("Please draw at least one polygon of interest first!")
    print("For more detailed information, read the README.md file.")
else:
    # Get all the area of interest for the analysis
    for feature in Map.draw_features:          
        geometry = ee.Geometry(feature.geometry())
        geometries_list.append(geometry)
        print("Information about the area of interest acquired correctly!")

# Combine the geometries into a single geometry (e.g., a MultiPolygon if they are polygons)
combined_geometry = ee.Geometry.MultiPolygon(geometries_list)

Information about the area of interest acquired correctly!


In [88]:
# Filter Sentinel-2 image collection
try:
    sentinel_images = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start_date, end_date)
        .filterBounds(combined_geometry)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    )
    print("Sentinel-2 images acquired correctly!")

except Exception as e:
    print("An error occurred while trying to acquire the Sentinel-2 images.")
    print(f"Error: {e}")

Sentinel-2 images acquired correctly!


#### NDVI

NDVI value ranges between -1.0 and +1.0. Generally speaking, NDVI shows a functional relationship with vegetation properties (e.g. biomass). NDVI is directly related to the photosynthetic capacity and energy absorption of plant canopies. The NDVI is calculated from these individual measurements as follows:

`NDVI= (NIR-Red) \ (NIR+Red)`

In [89]:
# Function to calculate NDVI using NIR (B8) and Red (B4) bands
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return ndvi

In [90]:
# Get the list of dates from the image collection
dates = sentinel_images.aggregate_array("system:time_start").getInfo()
dates = [ee.Date(date).format("YYYY-MM-dd").getInfo() for date in dates]
unique_dates = sorted(set(dates))
# unique_dates[0] = "2024-07-01"
print(f"Unique dates: {unique_dates}")


# Function to convert date format
def convert_date_format(date):
    year, month, day = date.split("-")
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    month_abbr = months[int(month) - 1]
    return f"{day} {month_abbr}"

# Create a mapping from original date format to formatted date
# Needed because we want to visualize DD-MMM format on the slider, but
# the Earth Engine date format is YYYY-MM-DD
date_mapping= {convert_date_format(date): date for date in unique_dates}

# Apply the conversion to the unique_dates list
formatted_dates = list(date_mapping.keys())
# formatted_dates[0] = "01 Jul"
print(f"Formatted dates: {formatted_dates}")

date_slider = widgets.SelectionSlider(
    options=formatted_dates,
    value=formatted_dates[0],  # default value
    description="",
    continuous_update=False,
    orientation="horizontal",
    readout=False,  # Hide the default readout
    style={"description_width": "initial"},
    layout=widgets.Layout(width="100%"),
)

Unique dates: ['2024-06-03', '2024-06-08', '2024-06-18', '2024-06-28', '2024-07-08', '2024-07-13', '2024-07-18']
Formatted dates: ['03 Jun', '08 Jun', '18 Jun', '28 Jun', '08 Jul', '13 Jul', '18 Jul']


In [92]:
import ipywidgets as widgets
from IPython.display import display

# Checkboxes to control NDVI and NDMI layer visibility
ndvi_checkbox = widgets.Checkbox(
    value=False, description="NDVI", disabled=False, indent=False
)

ndmi_checkbox = widgets.Checkbox(
    value=False, description="NDMI", disabled=False, indent=False
)

# Descriptions above the checkboxes
ndvi_label = widgets.HTML("<b>Health Indices</b>")
ndmi_label = widgets.HTML("<b>Water Stress Indices</b>")

# Combine labels and checkboxes in a VBox for each
ndvi_container = widgets.VBox([ndvi_label, ndvi_checkbox])
ndmi_container = widgets.VBox([ndmi_label, ndmi_checkbox])

# Display checkboxes in a single line
checkbox_crop_filter_container = widgets.HBox(
    [ndvi_container, ndmi_container]
)


In [93]:
# Callback function to update the map layer based on selected date
def on_date_change(change):
    
    if change["new"]:

        # Get the value of the selected date from the widget
        selected_date = date_slider.value

        if ndvi_checkbox.value:
            # get the date in the format YYYY-MM-DD for the Earth Engine filter
            date_ee_format = date_mapping[selected_date]

            selected_image = sentinel_images.filterDate(
                date_ee_format, ee.Date(date_ee_format).advance(1, "day")
            ).first()
            # clip the image to the combined geometry
            selected_image = selected_image.clip(combined_geometry)
            # compute the ndvi for the selected image
            ndvi_image = calculate_ndvi(selected_image)

            Map.remove_layer("NDVI")

            ndvi_params = {"min": 0, "max": 1, "palette": ["red", "yellow", "green"]}
            Map.addLayer(ndvi_image, ndvi_params, "NDVI")

            #Map.centerObject(combined_geometry)

            Map.add_colorbar(ndvi_params, label="Vegetation Health Index", layer_name="NDVI")

        else:
            Map.remove_layer("NDVI")

        #TODO: Add NDMI layer

# Create date marker divs
date_marker_divs = [widgets.HTML(value=f"<div>{date}</div>") for date in formatted_dates]

# Callback function to monitor the ndvi_checkbox checkbox status
def on_ndvi_checkbox_change(change):
    if change["new"] == True:
        # check if the checkbox is checked
        if date_slider.value:
            # take the date from the slider
            selected_date = date_slider.value
            date_ee_format = date_mapping[selected_date]

            selected_image = sentinel_images.filterDate(
                date_ee_format, ee.Date(date_ee_format).advance(1, "day")
            ).first()
            # clip the image to the combined geometry
            selected_image = selected_image.clip(combined_geometry)
            # compute the ndvi for the selected image
            ndvi_image = calculate_ndvi(selected_image)

            Map.remove_layer("NDVI")

            ndvi_params = {"min": 0, "max": 1, "palette": ["red", "yellow", "green"]}
            Map.addLayer(ndvi_image, ndvi_params, "NDVI")

            #Map.centerObject(object=combined_geometry)

            Map.add_colorbar(ndvi_params, label="Vegetation Health Index", layer_name="NDVI")

    else:
        Map.remove_layer("NDVI")

In [96]:
# Observe the change in dropdown value
date_slider.observe(on_date_change, names="value")
ndvi_checkbox.observe(on_ndvi_checkbox_change, names="value")

# Create a container for the slider and date markers
date_slider_container = widgets.VBox(
    [
        widgets.HBox([date_slider], layout=widgets.Layout(width="100%")),
        widgets.HBox(
            date_marker_divs,
            layout=widgets.Layout(justify_content="space-between", width="100%"),
        ),
    ]
)

# Trigger the initial update to set the active date marker
# update_date_markers({"new": date_slider.value})

# Create a container for the map and the slider
container = widgets.VBox(
    [checkbox_crop_filter_container, Map, date_slider_container]
)

In [97]:
display(container)